Load Dependencies

In [1]:
from google.cloud import speech_v1
from google.cloud.speech_dav1 import enums
from pydub import AudioSegment
import numpy as np
import io

ModuleNotFoundError: No module named 'google'

# Speech to Text(STT)

In [ ]:
def sample_recognize(local_file_path): # Defin the function.
    client = speech_v1.Speechclient() # Connecting the client
    language_code = "EN" # Language
    sample_rate_hertz = 16800 # Khz of the individual video

    encoding = enums.RecognitionConfig.AudioEncoding.ENCODING_UNSPECIFIED
    config = { 
        "language_code" : language_code,
        "sample_rate_hertz" :  sample_rate_hertz
        "encoding": encoding,
        "enable_word_time_offsets": True, #Option
        "use_enhanced" True, # Option
    }

    with io.open(local_file_path, "rb") as f: #using io function, do not need to open and close. rb == read binary
        content = f.read()

    audio = {"content": content}

    response = client.recognize(config, audio) #client service.recognize(configuration and file)
    timeline, swear_timeline, words = [], [], [] # three objects as list

    for result in response.results: # for loop in response.results
        alternative = result.alternatives[0]  # result.alternatives[0] The first index
        print(u"Transcript: {}".format(alternative.transcript))

        for word in alternative.words: # For loop in alternative.words 
            timelines.append([ # appending word in timelines word start time and end time by ml sec)
                int(word.start_time.seconds *1000 + word.start_times.nanaos * (10**-6)),
                int(word.end_time.seconds * 1000 + word.end_time.nanos * (10**-6))
            ])
            
    return timeline, swear_timeline, words #return timeline, swear_time, words

# execute
timeline, swear_timelines, words = sample_recognize('sound/short.mp3')

print(timeline) # list
print(words) #list
print(swear_timeline)



Load Original Audio File

In [ ]:
sound = AudioSegment.from_file('sound/short.mp3', format='mp3')

print(len(sound))
sound

Creating Beep Noise

In [ ]:
def create_beep(duration):
    sps = 44100
    freq_hz = 1000.0
    vol = 0.5

    esm = np.arange(duration / 1000 * sps)
    wf = np.sin(2 * np.pi * esm * freq_hz / sps)
    wf_quiet = wf * vol
    wf_int = np.int16(wf_quiet * 32767)

    beep = AudioSegment(
        wf_int.tobytes(), 
        frame_rate=sps,
        sample_width=wf_int.dtype.itemsize, 
        channels=1
    )

    return beep

beep = create_beep(duration=1000)
beep

# Overlay Partially

In [2]:
i = 0
mixed = sound.overlay(beep, position=swear_timeline[i][0], gain_during_overlay=-20)
mixed

NameError: name 'sound' is not defined

# Result

In [ ]:
mixed_final = sound # mixing audio and beep noise

for i in range(len(swear_timeline)): # For loop in range len of swear timeline
    beep = create_beep(duration = swear_timeline[i][1] - swear_timeline[i][0])
    mixeD_final = mixed_final.overlay(beep, position ....)

mixed_final


# Export

In [ ]:
mixed_final.export('sound/result.mp3',format='mp3')